Lecture 2: Basic Single & Multi-Table SQL
======================

In [1]:
%load_ext sql
%sql sqlite://

'Connected: @None'

Let's create a table, stuff it with data, and query it!

In [2]:
%%sql drop table if exists product;
create table product(
       pname        varchar primary key, -- name of the product
       price        money,               -- price of the product
       category     varchar,             -- category
       manufacturer varchar NOT NULL     -- manufacturer
);
insert into product values('Gizmo', 19.99, 'Gadgets', 'GizmoWorks');
insert into product values('PowerGizmo', 29.99, 'Gadgets', 'GizmoWorks');
insert into product values('SingleTouch', 149.99, 'Photography', 'Canon');
insert into product values('MultiTouch', 203.99, 'Household', 'Hitachi');

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Let's look at the products..

In [3]:
%sql select * from product;

 * sqlite://
Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks
SingleTouch,149.99,Photography,Canon
MultiTouch,203.99,Household,Hitachi


Some *terminology* for SQL.
--------------------------
* The _name_ of the table is product.
* Each row of the table is called a _row_ or a _tuple_. 
* Notice all tuples have the fields or _attributes_.
* The number of rows is called the _cardinality_ while the number of attributes is called the _arity_

Schema Conventions
-----------------
* The schema of product is written as follows:

> product(<u>pname</u>, price, category, manufacturer)

Underlining a set of attributes indicates that they form a _key_.

* In this case, pname is a key. If the product name was only unique for a given manufacturer, we'd write:

> product(<u>pname</u>, price, category, <u>manufacturer</u>)


Tables Explained
----------------
* A tuple = a record
  * Restriction: all attributes are of atomic type
  * There are many atomic data types in SQL engines, look [here](http://www.postgresql.org/docs/9.4/static/datatype.html) for example.


* A table = a (multi)-set of tuples
  * A multiset is like a list…
  * ... but a mutiset is unordered: 
    * no first(), no next(), no last().

# Outline
* Create a database -- done!

* Simple querying -- now!

* Queries with more than one relation -- next!


# Let the (Simple) querying begin! 
* We'll introduce the basics of SQL by example.
* There are many good SQL tutorials on the web, this is intended to get you started.

> SELECT (attributes)<br>
> FROM (one or more tables)<br>
> WHERE (conditions)

This is the simple SELECT-FROM-WHERE (SFW) block. Let's see some examples!

In [4]:
%%sql
SELECT * from Product 
WHERE category='Gadgets' and price > 20.0;

 * sqlite://
Done.


pname,price,category,manufacturer
PowerGizmo,29.99,Gadgets,GizmoWorks


Let's give an example of *projection*, i.e., we only retain some attributes from the query. 

In [5]:
%%sql 
SELECT Pname, Price, Manufacturer
FROM Product;

 * sqlite://
Done.


pname,price,manufacturer
Gizmo,19.99,GizmoWorks
PowerGizmo,29.99,GizmoWorks
SingleTouch,149.99,Canon
MultiTouch,203.99,Hitachi


* The output is *still* a table, and its schema is 
> Answer(pname, price, manufacturer)

* Of course, we can combine selection and projection.

In [6]:
%%sql
SELECT Pname, Price, Manufacturer
FROM Product
WHERE category='Gadgets';

 * sqlite://
Done.


pname,price,manufacturer
Gizmo,19.99,GizmoWorks
PowerGizmo,29.99,GizmoWorks


The output of a query on a table is again a table 
----------------------------------------------
* This is because the query language is *compositional*
* The output of a query really is a table!
* look at this crazy query, what does it ask for?

In [7]:
%%sql
SELECT * FROM Product;

SELECT
    p.manufacturer, p.pname, p.price
FROM 
    (SELECT distinct p0.Manufacturer
     FROM Product p0
     WHERE p0.price < 20.00) cp, -- this is a nested query!
    Product p
WHERE 
    p.manufacturer = cp.manufacturer and p.price > 20.00

 * sqlite://
Done.
Done.


manufacturer,pname,price
GizmoWorks,PowerGizmo,29.99


Details on SQL
--------------

* Some elements are case insensitive (think: program):
  * Same: SELECT  Select  select
  * Same: Product   product
  * Different: ‘Seattle’  ‘seattle’
  

* Constants (single quotes)
  * ‘abc’  - yes
  * “abc” - no


LIKE
====

The LIKE operator is to search strings, perhaps with wildcards. Format is:
    
> SELECT *
> FROM Products
> WHERE pname like '%gizmo%'

* % matches any number of characters
* \_ matches one character
* The like operator is case sensitive


In [8]:
%%sql
SELECT *  FROM product
where pname LIKE '%Gizmo%'

 * sqlite://
Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks


Eliminating Duplicates
---------------------
* duplicates can sometimes be unwelcome or suprising. 
  * Recall tables are _multisets_!

In [9]:
%sql SELECT category from product;

 * sqlite://
Done.


category
Gadgets
Gadgets
Photography
Household


In [10]:
%%sql 
-- easy to remove duplicates, use the distinct keyword
SELECT DISTINCT category from product;

 * sqlite://
Done.


category
Gadgets
Photography
Household


Ordering the results
---------------------
* Sometimes you want the results ordered, let's see some examples!


In [11]:
%%sql
-- sometimes we want to order the results.
-- order by is ascending by default!
SELECT   pname, price, manufacturer
FROM     Product
WHERE    price > 50
ORDER BY  price, pname

 * sqlite://
Done.


pname,price,manufacturer
SingleTouch,149.99,Canon
MultiTouch,203.99,Hitachi


In [12]:
%%sql
-- sometimes we want to order the results.
-- can order like so, each component individually
SELECT   price, manufacturer
FROM     Product
-- the order is "dictionary order" in the clause.
ORDER BY   manufacturer ASC, price DESC

 * sqlite://
Done.


price,manufacturer
149.99,Canon
29.99,GizmoWorks
19.99,GizmoWorks
203.99,Hitachi


Joins!
------
Let's illustrate some more complex queries that join two tables together.

* Consider a table of companies, stock price, and HQ country.
> company(<u>cname</u>, stockprice, country)
 
* we'll then revist products and introduce some consistency requirements 

In [13]:
%%sql
drop table if exists product; -- This needs to be dropped if exists, see why further down!
drop table if exists company;
create table company (
    cname varchar primary key, -- company name uniquely identifies the company.
    stockprice money, -- stock price is in money 
    country varchar); -- country is just a string
insert into company values ('GizmoWorks', 25.0, 'USA');
insert into company values ('Canon', 65.0, 'Japan');
insert into company values ('Hitachi', 15.0, 'Japan');

 * sqlite://
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [14]:
%sql select * from company;

 * sqlite://
Done.


cname,stockprice,country
GizmoWorks,25,USA
Canon,65,Japan
Hitachi,15,Japan


Foreign Key Constraints
-----------------------
* Suppose that we want to create a products table

> Product(pname, price, category, manufacturer)

* Something is odd here: We can have manufacturers that sell products but don't occur in our company table!
* To protect against, this we introduce _foreign keys_ 

We say the company name in products _refers_ to cname in company. Let's do it in SQL! the key statement below is:

> foreign key (manufacturer) references company(cname)

  * Note that cname must be a key in company! 
  * Keys and Foreign keys come up _all_ the time. 
    * PKs and FKs are common (others, less so)

In [15]:
%%sql drop table if exists product;
pragma foreign_keys = ON; -- WARNING by default off in sqlite
create table product(
       pname varchar primary key, -- name of the product
       price money, -- price of the product
       category varchar, -- category
       manufacturer varchar, -- manufacturer
       foreign key (manufacturer) references company(cname));

insert into product values('Gizmo', 19.99, 'Gadgets', 'GizmoWorks');
insert into product values('PowerGizmo', 29.99, 'Gadgets', 'GizmoWorks');
insert into product values('SingleTouch', 149.99, 'Photography', 'Canon');
insert into product values('MultiTouch', 203.99, 'Household', 'Hitachi');

 * sqlite://
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Indeed foreign keys are a _constraint_ 
> What happens if we introduce a company name not in our table?


In [17]:
try:
    %sql insert into product values('MultiTouch', 203.99, 'Household', 'Google');
except Exception as e:
    print(e) 
    print("Rejected!") 

 * sqlite://
(sqlite3.IntegrityError) UNIQUE constraint failed: product.pname
[SQL: insert into product values('MultiTouch', 203.99, 'Household', 'Google');]
(Background on this error at: http://sqlalche.me/e/gkpj)
Rejected!


In [18]:
%%sql
-- the update is rejected!
select * from product;

 * sqlite://
Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks
SingleTouch,149.99,Photography,Canon
MultiTouch,203.99,Household,Hitachi


Foreign Keys and Delete
=============

* What happens if we delete a company? Three options:
  * Disallow the delete. (default)
  * Remove all products (add "`on delete cascade`")
  * A third variant due to NULL
  

**First option (default)- delete is disallowed**

In [21]:
try:
    %sql delete from company where cname = 'Hitachi';
except Exception as e:
    print(e)
    print("Disallowed!")

 * sqlite://
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: delete from company where cname = 'Hitachi';]
(Background on this error at: http://sqlalche.me/e/gkpj)
Disallowed!


**Second option: remove all products belonging to the company we delete**

Try adjusting the foreign key constraint clause when you create the products table as follows:
> foreign key (manufacturer) references company(cname) on delete cascade

Now, when a company row is deleted, all of the products linked by the foreign key constraint will be deleted as well.

Joins!
------
> Product (<u>pname</u>,  price, category, manufacturer)<br>
> Company (<u>cname</u>, stockPrice, country)

The query we want to answer is:

> Find all products under $200 manufactured in Japan;
> return their names and prices. 

Notice products don't have a location and manufacturers don't have price. Need info in _each_ of the tables.

In [22]:
%%sql
SELECT pname, price
FROM product, company
where manufacturer=cname and country='Japan' and price <= 200;

 * sqlite://
Done.


pname,price
SingleTouch,149.99


Let's see how to write this join in a modular way.

In [23]:
%%sql -- Part 1: Set of Japanese companies.
SELECT distinct cname -- do we need distinct?
from company where country='Japan';

 * sqlite://
Done.


cname
Canon
Hitachi


In [24]:
%%sql -- Part 2: Cheap Products (under $200)
select distinct pname, price, manufacturer
from product
where price <= 200;

 * sqlite://
Done.


pname,price,manufacturer
Gizmo,19.99,GizmoWorks
PowerGizmo,29.99,GizmoWorks
SingleTouch,149.99,Canon


In [25]:
%%sql -- combine them with nested SFW queries... this is a cross product?
SELECT * 
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 200) CheapProducts,
  (SELECT DISTINCT cname
   FROM company
   WHERE country='Japan') JapaneseProducts;

 * sqlite://
Done.


pname,price,manufacturer,cname
Gizmo,19.99,GizmoWorks,Canon
Gizmo,19.99,GizmoWorks,Hitachi
PowerGizmo,29.99,GizmoWorks,Canon
PowerGizmo,29.99,GizmoWorks,Hitachi
SingleTouch,149.99,Canon,Canon
SingleTouch,149.99,Canon,Hitachi


In [26]:
%%sql
-- Combine them as a join!
SELECT DISTINCT pname, price
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 200) CheapProducts,
  (SELECT distinct cname
   FROM company
   WHERE country='Japan') JapaneseProducts
WHERE cname = manufacturer;

 * sqlite://
Done.


pname,price
SingleTouch,149.99


Takeways
--------
* There are potentially _many logically equivalent ways_ to write a query
    * This fact will be used later by the query optimizer and in homework!
    * On exams, write the simplest thing (break it down in parts?)

Duplicate answers from join
--------------------------

Note that we can get duplicate answers from a join...

In [27]:
%%sql -- duplicate answer
SELECT Country
FROM Product, Company
WHERE  Manufacturer=CName AND Category='Gadgets';

 * sqlite://
Done.


country
USA
USA
